In [1]:
from QCutils import *
import numpy as np
import numpy as jnp

In [2]:
N = 6
beta = 1
J = 0.5
h = 0.3
sigma_z = np.array([[1,0],[0,-1]])
sigma_x = np.array([[0,1],[1,0]])
sigma_y = np.array([[0,1],[1,0]])

spin = np.array([0,1])
spin_up = np.array([[1],[0]])
spin_down= np.array([[0],[1]])

In [3]:
#def createH(J,h)

H = jnp.zeros((2**N,2**N))

initLattice = jnp.kron(sigma_z,sigma_z)
for i in range(2,N):
    initLattice = jnp.kron(initLattice,jnp.eye(2))

H += - J * initLattice

for lattice_point in range(1,N-1):
    curr = jnp.eye(2)
    for i in range(1,lattice_point):
        curr = jnp.kron(curr,jnp.eye(2))
    curr = jnp.kron( jnp.kron(curr,sigma_z),sigma_z)
    for i in range(lattice_point+2,N):
        curr = jnp.kron(curr,jnp.eye(2))
    
    assert curr.shape[0] == H.shape[0]
    
    H += -J * curr


initLattice = sigma_x
for i in range(1,N):
    initLattice = jnp.kron(initLattice,jnp.eye(2))

H += - h * initLattice

for lattice_point in range(1,N-1):
    curr = jnp.eye(2)
    for i in range(1,lattice_point):
        curr = jnp.kron(curr,jnp.eye(2))
    curr = jnp.kron(curr,sigma_x)
    for i in range(lattice_point+1,N):
        curr = jnp.kron(curr,jnp.eye(2))
    
    assert curr.shape[0] == H.shape[0]
    
    H += -h * curr

H = jnp.array(H,dtype=jnp.complex128)

In [4]:
formator = '{0:' + '0' + str(N)  +'b}'

state_in_str = [formator.format(i) for i in range(2**N)]

def state_to_vec(s):
    
    if s[0] == '1':
        state = spin_up
        state_mat = spin_up
    else:
        state = spin_down
        state_mat = spin_down
    
    for curr in s[1:]:
        if curr == '1':
            state = np.kron(state,spin_up)
            state_mat = np.hstack((state_mat,spin_up))
        else:
            state = np.kron(state,spin_down)
            state_mat = np.hstack((state_mat,spin_down))
    
    return state,state_mat

stateDict = dict()
for s in state_in_str:
    stateDict[s] = state_to_vec(s)[0]


In [5]:
q = QuantumRegister(N)
c = ClassicalRegister(N)
qc = QuantumCircuit(q,c)

thetas = np.random.rand(3 * N)
for i in range(N):
    qc.rx(thetas[i] * np.pi/2,i)
    qc.ry(thetas[i] * np.pi/2,i)
    qc.rz(thetas[i] * np.pi/2,i)

qc.measure(q,c)
backend = Aer.get_backend('qasm_simulator')
job = execute(qc, backend, shots=1000)
QCresult = job.result().get_counts(qc)

In [6]:
def FCostFunc(QCresult):
    estF = 0
    for key in QCresult.keys():
        state = stateDict[key]
        prob = QCresult[key] / QuCircWithGrad.simShots
        estF += prob * (np.log(prob) + beta * state.T @ H @ state)
    return estF[0][0]


In [7]:
test1 = QuCircWithGrad(N)

for i in range(N):
    test1.add_gate('Rx',i)

for i in range(4):
    for i in range(N-1):
        test1.add_gate('Rxx',(i,i+1))
        test1.add_gate('Ryy',(i,i+1))
        test1.add_gate('Rzz',(i,i+1))

    for j in range(N-1):
        i = (N-2 - i)
        test1.add_gate('Rxx',(i,i+1))
        test1.add_gate('Ryy',(i,i+1))
        test1.add_gate('Rzz',(i,i+1))

In [8]:
test1.setCostFun(FCostFunc)
test1.construct()
test1.calc(),test1.getThetaGrad()

/home/qwe/MLPhys/QC/pure/QCutils.py:97: ComplexWarning: Casting complex values to real discards the imaginary part
  thetaGrad[i] = grad1


((-3.6352642524765484+0j),
 array([-0.08348307,  0.19248107,  0.05552566, -0.07757359,  0.00179987,
        -0.13073542,  0.15359113, -0.01545523, -0.06794485, -0.12005781,
        -0.11582725,  0.07294065, -0.16237886, -0.02708733,  0.11779016,
         0.05833866,  0.19868836,  0.08594824,  0.1405052 , -0.08573416,
        -0.03087916,  0.1245387 , -0.00421854, -0.02670766,  0.18899885,
        -0.04953069, -0.02045867,  0.08101231, -0.03079402,  0.00221324,
         0.15019174,  0.00575936,  0.01146705,  0.09753987,  0.03585808,
        -0.02949723,  0.0731844 , -0.02723611, -0.00276052, -0.02498571,
        -0.12281997,  0.10138373, -0.07145718, -0.22798783, -0.1850007 ,
         0.0551506 , -0.03379632, -0.01592876, -0.0376846 ,  0.09818481,
        -0.06217582,  0.007696  , -0.05286841,  0.17694144,  0.01169768,
         0.02573217, -0.11680773, -0.0121377 , -0.03233453,  0.20140669,
        -0.01311284,  0.03250005, -0.06562012, -0.04795133, -0.03560628,
         0.19315571,  0.

In [9]:
def ValandGrad(thetas):
    test1.updateThetas(thetas)
    return test1.calc(), test1.getThetaGrad()

In [10]:
import scipy.optimize
initThetas = 2* np.pi * np.random.rand(test1.size())
results = scipy.optimize.minimize(ValandGrad, initThetas,
                                  method='L-BFGS-B', jac=True)
print("success:", results.success, "\nniterations:", results.nit, "\nfinal loss:", results.fun)

/home/qwe/miniconda3/envs/qi/lib/python3.9/site-packages/scipy/optimize/_lbfgsb_py.py:353: ComplexWarning: Casting complex values to real discards the imaginary part
  _lbfgsb.setulb(m, x, low_bnd, upper_bnd, nbd, f, g, factr,


success: True 
niterations: 5 
final loss: (-4.38049675102553+0j)


In [11]:
from scipy.linalg import expm
-np.log(np.trace(expm(- beta * H)))

(-4.9530060181077005-0j)

In [21]:
learningRate = 0.05
maxStep = 100
record = np.zeros(maxStep)

thetas = 2* np.pi * np.random.rand(test1.size())

for i in range(maxStep):
    test1.updateThetas(thetas)
    record[i] = test1.calc()
    dTheta = test1.getThetaGrad()
    thetas -= learningRate * dTheta

/tmp/ipykernel_10766/1987415637.py:9: ComplexWarning: Casting complex values to real discards the imaginary part
  record[i] = test1.calc()


In [16]:
record[record != 0][0]

-1.9857282192311847

In [9]:
r = 0.5
circList = test1.genGradCircList()
thetaGrad = np.zeros(test1.size())

In [10]:
circPlus, circMinus = circList[1]

In [15]:
grad1 = r * (circPlus.calc() - circMinus.calc())
grad1[0][0]

(-0.276411996093495+0j)

In [11]:
circPlus, circMinus = circList[1]
grad1 = r * (circPlus.calc() - circMinus.calc())
thetaGrad[1] = grad1


In [9]:
lst = test1.genGradCircList()

In [12]:
len(lst)
circPlus, circMinus = lst[0]

In [16]:
circMinus.calc()
grad = 0.5 * (circPlus.calc() - circMinus.calc())
grad

/Users/qwe/miniforge3/envs/qi/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/qwe/miniforge3/envs/qi/lib/python3.10/site-packages/numpy/linalg/linalg.py:2146: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)


array([[-0.01312868+0.j]])

In [ ]:
test1.getThetaGrad()